# От множества к пространству

Чем «пространство» отличается от «множества»? Если совсем на пальцах. Тем, что над его элементами определены операции. Соответственно, если мы просто «напишем» класс, то определим тип данных, элемент множества. А если реализуем для него операции — определим пространство.

Тяжело найти что-то, чего нет в стандартной библиотеке. А тем более во внешних...

![](https://imgs.xkcd.com/comics/python.png)

Давайте считать, что кольцо многочленов не реализовано. А ведь полезная вещь! А как его реализовать? А очень просто: [вот тут всё написано](https://docs.python.org/3.7/reference/datamodel.html#emulating-numeric-types)!

In [3]:
class QuotientRingError(ValueError):
    pass


class QuotientRing:
    """ """
    
    def __init__(self, arg: 'int|QuotientRing' = 0, mod: 'int' = None):
        """
        """
        if isinstance(arg, QuotientRing):
            self.n = arg.n
            self.mod = arg.mod
        elif isinstance(arg, int):
            if mod != None:
                self.n = arg % mod
                self.mod = mod
            else:
                raise QuotientRingError("Modulo not specified")
        else:
            raise QuotientRingError("You are trying to create QuotientRing from " + repr(arg))
        
    def __str__(self) -> 'str':
        return str(self.n)
    
    def __eq__(self, other: 'QuotientRing|int') -> 'bool':
        if isinstance(other, QuotientRing):
            return self.n == other.n
        elif isinstance(other, int):
            return self.n == (other % self.mod)
        else:
            raise QuotientRingError("Can't say if QuotientRing is equal to " + str(type(other)))
        
    
    def __add__(self, other: 'QuotientRing|int') -> 'QuotientRing':
        if isinstance(other, QuotientRing):
            return QuotientRing((self.n + other.n) % self.mod, self.mod)
        elif isinstance(other, int):
            return QuotientRing((self.n + other) % self.mod, self.mod)
        else:
            raise QuotientRingError("Can't add not QuotientRing " + str(type(other)))
           
    def __radd__(self, other: 'QuotientRing|int') -> 'QuotientRing':
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self) -> 'QuotientRing':
        return QuotientRing(-self.n % self.mod, self.mod)

    
    def __sub__(self, other: 'QuotientRing|int') -> 'QuotientRing':
        if isinstance(other, QuotientRing):
            return self.__add__(other.__neg__())
        elif isinstance(other, int):
            return self.__add__(QuotientRing(other, self.mod).__neg__())
        else:
            raise QuotientRingError("You are trying to subtract not QuotientRing " + str(type(other)))
    
    def __rsub__(self, other: 'QuotientRing|int') -> 'QuotientRing':
        return self.__neg__().__add__(other)

    
    def __mul__(self, other: 'QuotientRing|int') -> 'QuotientRing':
        if isinstance(other, QuotientRing):
            return QuotientRing((self.n * other.n) % self.mod, self.mod)
        elif isinstance(other, int):
            return QuotientRing((self.n * other) % self.mod, self.mod)
        else:
            raise QuotientRingError("Can't add multipy QuotientRing by " + str(type(other)))
        
        
    def __rmul__(self, other: 'QuotientRing|int') -> 'QuotientRing':
        return self.__mul__(other)

Ну а теперь давайте посмотрим его в деле!

In [5]:
Z0 = QuotientRing(0, 5)
Z1 = QuotientRing(1, 5)
Z2 = QuotientRing(2, 5)
Z4 = QuotientRing(4, 5)

print(Z0 * Z4)
print(Z2 * Z4)
print(Z2 + Z4)
print(Z2 - Z4)
print(Z1 == Z4)
print(Z2 * Z2 == Z4)
print(Z4 * 4)

0
3
1
3
False
True
1


## Задание

Придумать и реализовать какой-нибудь свой тип данных с операциями. Вот идеи:

* числа по модулю;
* обычные числа, но с ошибками при вычислениях;
* [кватеринионы](https://en.wikipedia.org/wiki/Quaternion);
* можно матрицы (в т.ч. вектора) — они и так в NumPy есть, но всё равно же интересно;
* да тот же многочлен, но при помощи [`defaultdict`](https://docs.python.org/3.7/library/collections.html#collections.defaultdict) — ему не повредит;
* что угодно на свой вкус, над чем можно определить арифметику или её подобие.